In [1]:

    var sparkConf = new SparkConf().setAppName("DecisionTreeRegressionnExample");
    var sc = new SparkContext(sparkConf);


 Load and parse the data file.


In [2]:
var datapath =  "../data/mllib/sample_libsvm_data.txt";
    var data = MLUtils.loadLibSVMFile(sc, datapath);


 Split the data into training and test sets (30% held out for testing)


In [3]:
    var splits = data.randomSplit([0.7, 0.3]);
    var trainingData = splits[0];
    var testData = splits[1];


 Set parameters.
  Empty categoricalFeaturesInfo indicates all features are continuous.


In [4]:
    var categoricalFeaturesInfo = {};
    var impurity = "variance";
    var maxDepth = 5;
    var maxBins = 32;


 Train a DecisionTree model.


In [5]:
    var model = DecisionTree.trainRegressor(trainingData,
        categoricalFeaturesInfo, impurity, maxDepth, maxBins);


 Evaluate model on test instances and compute test error


In [6]:

    var predictionAndLabel = testData.mapToPair(function (labeledPoint, model) {
        return new Tuple(model.predict(labeledPoint.getFeatures()), labeledPoint.getLabel());
    }, [model]);

    var testMSE = predictionAndLabel.map(function (tuple2) {
            var diff = parseFloat(tuple2[0] - tuple2[1]);
            return diff * diff;
        }).reduce(function (a, b) {
            return a + b;
        }) / data.count();

    var ret = {};
    ret.testMSE = testMSE;
    ret.model = model;
var result = ret;
    print("Test Mean Squared Error: " + result.testMSE);
    print("Learned regression tree model:\n" + result.model.toDebugString());


 Save and load model


In [7]:
    result.model.save(sc, "target/tmp/myDecisionTreeRegressionModel");
    var sameModel = DecisionTreeModel
        .load(sc, "target/tmp/myDecisionTreeRegressionModel");

    sc.stop();
